In [39]:
import collections
 
# import tensorflow as tf
import h5py
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
import os
import pybedtools
import torch
# import h5sparse
import datetime
import logging

import tempfile
from scipy import stats
import argparse
import sys

# Files in repository
exec(open("./constants.py").read())
exec(open("./functions.py").read())
exec(open("./generators.py").read())
exec(open("./models.py").read())
feature_path = '../data/feature_name'

_DEEPSEA_GENOME_REGIONS_FILENAME = '../data/allTFs.pos.bed'


In [4]:
########################## PARSE USER ARGUMENTS ###########################
label_path="/data/akmorrow/epitome_data/deepsea_labels_train/"

output_path="/home/eecs/akmorrow/epitome/out/Epitome/processed_datasets/TiPS_df_log_ratio_bkg.tsv"

deepsea_path="/data/akmorrow/epitome_data/deepsea_train/"

model_path='/data/akmorrow/epitome_data/saved_models/model__2019_04_01__14_48.ckpt'

peak_file="/home/eecs/akmorrow/epitome/data/test.bed"



#################### Functions ###############################

In [353]:

     
#################### Start code ##############################
     
# TODO AM 4/3/2019: make this work with just the labels so it runs faster
# all_data = load_deepsea_data_allpos_file(deepsea_path)
# train_data, valid_data, test_data = load_deepsea_label_data(label_path)
     
matrix, cellmap, assaymap = get_assays_from_feature_file(feature_path=feature_path, 
                                  eligible_assays = None,
                                  eligible_cells = None, min_cells_per_assay = 2, min_assays_per_cell=5)
     
     
     


     

In [4]:
############## Load Model #######################
exec(open("./models.py").read())
exec(open("./generators.py").read())
model  = MLP(4, [100, 100, 100, 50], 
            tf.tanh, 
            train_data, 
            valid_data, 
            test_data, 
            [],
            gen_from_peaks, 
            matrix,
            assaymap,
            cellmap,
            shuffle_size=2, 
            radii=[1,3,10,30])

model.restore(model_path)
     

eval cell types ['T-47D', 'SK-N-SH_RA', 'PANC-1', 'NB4', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HUVEC', 'HCT-116', 'H1-hESC', 'GM12892', 'GM12891', 'GM12878', 'A549']
using ['T-47D', 'SK-N-SH_RA', 'PANC-1', 'NB4', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HUVEC', 'HCT-116', 'H1-hESC', 'GM12892', 'GM12891', 'GM12878', 'A549'] as labels for mode Dataset.TRAIN
using ['T-47D', 'SK-N-SH_RA', 'PANC-1', 'NB4', 'MCF-7', 'K562', 'HepG2', 'HeLa-S3', 'HUVEC', 'HCT-116', 'H1-hESC', 'GM12892', 'GM12891', 'GM12878', 'A549'] as labels for mode Dataset.VALID
using [] as labels for mode Dataset.TEST
Error: no data, local variable 'x' referenced before assignment
INFO:tensorflow:Scale of 0 disables regularizer.


/data/akmorrow/miniconda3/envs/EpitomeEnv/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Initializing variables
INFO:tensorflow:Starting Training
INFO:tensorflow:Restoring parameters from /data/akmorrow/epitome_data/saved_models/model__2019_04_01__14_48.ckpt
INFO:tensorflow:Starting Training


In [64]:
_DEEPSEA_GENOME_REGIONS_FILENAME

'../data/allTFs.pos.bed'

In [111]:
positions = BedTool( "../data/allTFs.pos.bed")

test_bed_file_1 = "../data/test.bed"
test_bed_file_2 = "../data/test_without_scores.bed"
bed_file = test_bed_file_1

In [103]:
class Region:
    """Genomic Region"""

    def __init__(self, chrom, start, end):
        self.chrom = chrom
        self.start = start
        self.end = end
        
    def __str__(self):
        return("%s:%d-%d" % (self.chrom, self.start, self.end))
    
    def overlaps(self, other, min_bp = 1):
        return(other.chrom == self.chrom and self.overlap([self.start, self.end], [other.start, other.end]) > min_bp)
    
    def overlap(self, interval1, interval2):
        """
        Computes overlap between two intervals
        """
        if interval2[0] <= interval1[0] <= interval2[1]:
            start = interval1[0]
        elif interval1[0] <= interval2[0] <= interval1[1]:
            start = interval2[0]
        else:
            return 0

        if interval2[0] <= interval1[1] <= interval2[1]:
            end = interval1[1]
        elif interval1[0] <= interval2[1] <= interval1[1]:
            end = interval2[1]
        else:
            return 0

        return(end - start)

    def greaterThan(self, other):
        return(self.chrom > other.chrom or ( self.chrom == other.chrom and self.start > other.end ))

In [163]:
%%time 

def bedFile2Vector(bed_file, all_pos_file, duplicate = True):
    """
    This function takes in a bed file of peaks and converts it to a vector or 0/1s that can be 
    uses as input into a model. Each 0/1 represents a region in the train/test/validation set from DeepSEA.

    Most likely, the bed file will be the output of the IDR function, which detects peaks based on the
    reproducibility of multiple samples.

    :param: bed_file: bed file containing peaks
    :param: all_pos_file: file from DeepSEA that specified  genomic region ordering
    :param: duplicate: duplicates the strands to match deepsea's dataset. This should eventually be removed, as we do not want duplication if no DNA sequence is used. 

    :return: vector containing the concatenated 4.4 million train, validation, and test data in the order
    that we have parsed train/test. And returns a bedtool of regions that do not have corresponding locations in 
    the training data. These can be used later to set to 0.

    """

    # load in bed file and tf pos file
    positions = BedTool(all_pos_file)
    idr_peaks = BedTool(bed_file)

    # min base pair overlap to consider peak within a training region
    bp_overlap = 100

    peak_vector=np.zeros(_TOTAL_POSFILE_REGIONS)

    # keeps track of which idr peaks were overlapping something in the positions file
    found = np.zeros(idr_peaks.count())

    n_peaks = idr_peaks.count()
    last_peak = idr_peaks[n_peaks-1]
    # index of first idr peak to look at
    first_peak_idx = 0

    for position_i, pos in enumerate(positions):

        region = Region(pos.chrom, pos.start, pos.end)

        for peak_i in range(first_peak_idx, n_peaks):
            peak = idr_peaks[peak_i]
            peak = Region(peak.chrom, peak.start, peak.end)
            
            # if region is past this peak
            if (region.greaterThan(peak)):
                break

            if (region.overlaps(peak, bp_overlap)):
                peak_vector[position_i] = 1
                found[peak_i] = 1
                

        if (region.greaterThan(last_peak)):
            break
            
    # indices of peaks that are missing, have no training positions overlapping
    missing_idx = np.where(found == 0)[0]
    
    # filter out rows that were not used for training (defined in constants.py)
    # also, add in fake negative strands that just duplicate the sets 
    ATAC_train = peak_vector[_TRAIN_REGIONS[0]:_TRAIN_REGIONS[1]+1]
    ATAC_valid = peak_vector[_VALID_REGIONS[0]:_VALID_REGIONS[1]+1]
    ATAC_test =  peak_vector[_TEST_REGIONS[0]:_TEST_REGIONS[1]+1]

    # need to duplicate results for forward/reverse strand
    if (duplicate):
        vector = np.concatenate([ATAC_train, ATAC_train, ATAC_valid, ATAC_valid, ATAC_test, ATAC_test], axis=0)
    else:
        vector = np.concatenate([ATAC_train, ATAC_valid, ATAC_test], axis=0)

    return vector, missing_idx




vec, missing = bedFile2Vector(test_bed_file_1 , _DEEPSEA_GENOME_REGIONS_FILENAME, duplicate = False)

assert(vec.sum() == 3)
assert(missing.shape[0] == 2)

CPU times: user 7.53 ms, sys: 8.67 ms, total: 16.2 ms
Wall time: 15.6 ms


In [180]:
%%time
with open("../data/allTFs.pos.bed") as f:
    content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    idr_peaks = list(map(lambda x: Region(x.split()[0], int(x.split()[1]), int(x.split()[2])), content))
idr_peaks


CPU times: user 6.73 s, sys: 341 ms, total: 7.07 s
Wall time: 7.09 s


In [194]:
%%time
# takes >1s a while if empty
it = filter(lambda x: x.overlaps(Region("chr11", 123, 1234)), idr_peaks)
next(it, None)

CPU times: user 1.61 s, sys: 6.89 ms, total: 1.62 s
Wall time: 1.62 s


In [282]:
idr_peaks_tmp = idr_peaks[0:3]
idr_peaks_tmp

In [354]:
%%time
# v2 simple

def bedFile2Vector(bed_file, all_pos_file, duplicate = True):
    """
    This function takes in a bed file of peaks and converts it to a vector or 0/1s that can be 
    uses as input into a model. Each 0/1 represents a region in the train/test/validation set from DeepSEA.

    Most likely, the bed file will be the output of the IDR function, which detects peaks based on the
    reproducibility of multiple samples.

    :param: bed_file: bed file containing peaks
    :param: all_pos_file: file from DeepSEA that specified  genomic region ordering
    :param: duplicate: duplicates the strands to match deepsea's dataset. This should eventually be removed, as we do not want duplication if no DNA sequence is used. 

    :return: vector containing the concatenated 4.4 million train, validation, and test data in the order
    that we have parsed train/test. And returns a zipped of regions in the bed file and whether or not that
    peak had an associated peak in the training set.

    """

    # load in bed file and tf pos file
    positions = BedTool(all_pos_file)
    
    
    idr_peaks = []
    
    # load in peaks as a list of regions
    with open(bed_file) as f:
        content = f.readlines()
        # you may also want to remove whitespace characters like `\n` at the end of each line
        init_idr_peaks = list(map(lambda x: Region(x.split()[0], int(x.split()[1]), int(x.split()[2])), content))
    
    idr_peaks = list(enumerate(init_idr_peaks))
    
    # min base pair overlap to consider peak within a training region
    bp_overlap = 100

    peak_vector=np.zeros(_TOTAL_POSFILE_REGIONS)

    n_peaks = len(idr_peaks)
    
    # keeps track of which idr peaks were overlapping something in the positions file
    found = np.zeros(n_peaks)
    
    def setFound(i):
        found[i]=1

    curr_chrom = "chr1"
    idr_peaks_for_chrom = list(filter(lambda x: x[1].chrom == curr_chrom, idr_peaks))
    
    for position_i, pos in enumerate(positions):
            

        region = Region(pos.chrom, pos.start, pos.end)
        
        if (region.chrom != curr_chrom):
            curr_chrom == region.chrom
            idr_peaks_for_chrom = list(filter(lambda x: x[1].chrom == curr_chrom, idr_peaks))

        # filter in idr_peaks that overlap
        # takes >1s a while if empty
        it = filter(lambda x: x[1].overlaps(region, bp_overlap), idr_peaks_for_chrom)
    
        # if peak exists, set peak_vector to 1
        x = next(it, None)
        if (x != None):
            peak_vector[position_i] = 1
            found[x[0]] = 1
            map(lambda x: setFound(x[0]), it)
            
        # remove elements that are behind position_i
        k = 0
        while (region.greaterThan(idr_peaks[k][1])):
            k += 1
            if (k >= len(idr_peaks)):
                break
        
        del idr_peaks[0:k]
        
        # once idr_peaks is empty, return
        if (len(idr_peaks) == 0):
            break
            
        if (position_i == 10000):
            print("processing through lines in allpos bed file %i" % position_i)

            
    # indices of peaks that are missing, have no training positions overlapping
#     missing_idx = np.where(found == 0)[0]
    
    # filter out rows that were not used for training (defined in constants.py)
    # also, add in fake negative strands that just duplicate the sets 
    ATAC_train = peak_vector[_TRAIN_REGIONS[0]:_TRAIN_REGIONS[1]+1]
    ATAC_valid = peak_vector[_VALID_REGIONS[0]:_VALID_REGIONS[1]+1]
    ATAC_test =  peak_vector[_TEST_REGIONS[0]:_TEST_REGIONS[1]+1]

    # need to duplicate results for forward/reverse strand
    if (duplicate):
        vector = np.concatenate([ATAC_train, ATAC_train, ATAC_valid, ATAC_valid, ATAC_test, ATAC_test], axis=0)
    else:
        vector = np.concatenate([ATAC_train, ATAC_valid, ATAC_test], axis=0)

    print(list(init_idr_peaks))
    return vector, zip(init_idr_peaks, found)




vec, missing = bedFile2Vector("../data/test.bed" , _DEEPSEA_GENOME_REGIONS_FILENAME, duplicate = False)

# print(vec, missing)
# assert(vec.sum() == 3)
# assert(missing.shape[0] == 2)

[<__main__.Region object at 0x1338f1550>, <__main__.Region object at 0x1338f1470>, <__main__.Region object at 0x1338f1400>]
CPU times: user 7.23 ms, sys: 9.86 ms, total: 17.1 ms
Wall time: 17.1 ms


In [323]:
list(missing)

[(<__main__.Region at 0x133711fd0>, 1.0),
 (<__main__.Region at 0x133714198>, 0.0),
 (<__main__.Region at 0x133714080>, 0.0)]

In [311]:
missing[0:10]

array([ 2,  3,  4,  7,  8,  9, 10, 11, 12, 13])

In [309]:
r1 = Region("chr1",237600,237800)
r2 = Region("chr1",237679,237837)
r2.overlaps(r1, 100)

True

In [105]:
x = Region("chr1",10000,10200)
y = Region("chr1",10900,11100)


# test overlaps
assert(x.overlaps(y, 100) == False)

# test greaterThan
assert(y.greaterThan(x))
assert(Region("chr2",10900,11100).greaterThan(Region("chr1",10900,11100)))

In [129]:
idr_peaks[2]

Interval(chr1:10900-11100)

In [135]:
%%time 

t = bedFile2Vector(test_bed_file_1, _DEEPSEA_GENOME_REGIONS_FILENAME, duplicate = True)
# assert(t.sum() == 3)

CPU times: user 15.4 ms, sys: 22.9 ms, total: 38.3 ms
Wall time: 37.4 ms


In [384]:


df = score_peak_file(test_bed_file_1)
assert(len(df.index) == BedTool(test_bed_file_1).count())
assert(df.iloc[0][3:].sum() > 0)
assert(df.iloc[1][3:].sum() == 0)
assert(df.iloc[2][3:].sum() == 0)

../data/test.bed
[<__main__.Region object at 0x1338cbef0>, <__main__.Region object at 0x1338cbc88>, <__main__.Region object at 0x1338cb588>]
finished loading peak file


In [385]:
df

,chrom,start,end,p300,c-Myc,c-Jun,c-Fos,Znf143,ZZZ3,ZNF274,...,BRF1,BRCA1,BHLHE40,BDP1,BCLAF1,BCL3,BCL11A,ATF3,ATF2,ARID3A
0,chr1,900,10600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,chr1,10000,10020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr1,10900,11100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [597]:
assert(len(df2.index) == BedTool(test_bed_file_2).count())
assert(df2.iloc[0][3:].sum() > 0)
assert(df2.iloc[1][3:].sum() == 0)

# Read in results

In [179]:
x = pd.read_csv("/home/eecs/akmorrow/epitome/out/Epitome/processed_datasets/TiPS_df_allpeaks.tsv", sep = "\t")
x

/data/akmorrow/miniconda3/envs/EpitomeEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,chrom,start,end,p300,c-Myc,c-Jun,c-Fos,Znf143,ZZZ3,...,BRF1,BRCA1,BHLHE40,BDP1,BCLAF1,BCL3,BCL11A,ATF3,ATF2,ARID3A
0,0,chr1,10006.0,10211.0,0.146164,0.023268,0.061632,0.105748,0.003715,0.000336,...,0.000002,0.002874,0.032585,0.000257,0.021576,0.058185,0.113354,0.009855,0.215326,0.023414
1,1,chr1,237679.0,237837.0,0.093725,0.016953,0.054390,0.052183,0.362818,0.000179,...,0.000001,0.002657,0.054064,0.000095,0.014170,0.006772,0.011306,0.015856,0.056915,0.080145
2,2,chr1,567302.0,567750.0,0.189387,0.087069,0.078237,0.136133,0.028411,0.000566,...,0.000004,0.009254,0.122103,0.000489,0.071005,0.089935,0.206177,0.024800,0.343330,0.055195
3,3,chr1,567902.0,568135.0,0.275853,0.333664,0.096479,0.095881,0.094278,0.004249,...,0.000022,0.054477,0.338882,0.000823,0.255527,0.138385,0.170492,0.062201,0.597193,0.148048
4,4,chr1,569816.0,570064.0,0.360922,0.160016,0.172295,0.191557,0.250050,0.001271,...,0.000009,0.042972,0.231974,0.000388,0.147663,0.089557,0.084279,0.083283,0.307296,0.139942
5,5,chr1,713757.0,714675.0,0.153875,0.030938,0.053084,0.138370,0.004587,0.000440,...,0.000003,0.003849,0.046384,0.000397,0.035805,0.088498,0.152427,0.010598,0.263041,0.025707
6,6,chr1,762439.0,763168.0,0.175443,0.048042,0.084316,0.156666,0.010772,0.000542,...,0.000004,0.006205,0.073558,0.000436,0.045495,0.096636,0.198118,0.017687,0.316192,0.038545
7,7,chr1,777136.0,777487.0,0.189659,0.086348,0.079223,0.138602,0.028387,0.000552,...,0.000004,0.009134,0.121101,0.000486,0.069681,0.089842,0.208117,0.024634,0.341943,0.055316
8,8,chr1,779572.0,779804.0,0.190362,0.096265,0.075100,0.128141,0.038505,0.000633,...,0.000005,0.011928,0.125521,0.000449,0.080214,0.085857,0.176559,0.026685,0.347812,0.057490
9,9,chr1,780100.0,780235.0,0.154862,0.031421,0.053221,0.140229,0.004660,0.000444,...,0.000003,0.003898,0.047311,0.000405,0.036514,0.089715,0.154779,0.010620,0.266532,0.025875
